<h3>Import needed libraries</h3>

In [2]:
import pydicom as dicom
import matplotlib.pyplot as plt
import os
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np

<h3>User input</h3>

In [3]:
parentDirectory = "C:/Users/jake/OneDrive/Documents/Jake/Academia/Graduate School/Research/Osseointegration/Methodv12/AMCOI25/DICOMImages/CT/"
saveDirectory = "C:/Users/jake/OneDrive/Documents/Jake/Academia/Graduate School/Research/Osseointegration/Methodv12/AMCOI25/FEA/"
patient = "AMCOI25"
identifier = "IM_"

<h3>Scan import</h3>

In [4]:
x = 512
y = 512

# path = "C:/Users/jake/OneDrive/Documents/Jake/Academia/Graduate School/Research/T1D/ROI Data/2001_R/S88340/S70"
path = parentDirectory
dir_list = os.listdir(path)
dir_list = [i for i in dir_list if identifier in i]

z = len(dir_list)
data = np.zeros((z, x, y))

for i in range(len(dir_list)):
    image_path = path + "/" + dir_list[i]
    ds = dicom.dcmread(image_path)
    j = ds.InstanceNumber - 1
    data[j,:,:] = ds.pixel_array

data = np.flip(data, axis=0)
# data = np.flip(data, axis=1)
# data = np.flip(data, axis=2)

<h3>Plot uncalibrated scan</h3>

In [5]:
def plot_func(j, a, b, L):
    l = int(L/2)
    plt.figure(figsize = (10, 10))
    plt.imshow(data[j,:,:], cmap = "bone")
    plt.axhline(a+0.5)
    plt.axvline(b+0.5)
    rectangle = plt.Rectangle((b-l, a-l), L, L)
    plt.gca().add_patch(rectangle)
    print(np.median(data[j,a-l:a+l,b-l:b+l]))

In [6]:
interact(plot_func, 
         j = widgets.IntSlider(value = 100, min = 0, max = z-1, step = 1),
         a = widgets.IntSlider(value = 260, min = 1, max = x, step = 1),
         b = widgets.IntSlider(value = 139, min = 1, max = y, step = 1),
         L = widgets.IntSlider(value = 30, min = 1, max = 100, step = 1))

interactive(children=(IntSlider(value=100, description='j', max=503), IntSlider(value=260, description='a', ma…

<function __main__.plot_func(j, a, b, L)>

In [ ]:
air = -985
fat = -138
muscle = 49

<h3>Calibrate scan and export</h3>

In [ ]:
pname = parentDirectory + patient + "_AFM_p.raw"
Ename = parentDirectory + patient + "_AFM_E.raw"

# for linear fit w/o bone
vals = [air, fat, muscle]
ref_vals = [-807, -85, 31]
fit = np.polyfit(vals, ref_vals, deg = 1)
p = np.poly1d(fit)
BMD = p(data)
p = 0.0012*BMD + 0.17
p[p <= 0.0] = 0

E = (6850*(np.power(p, 1.49)))*1.28
E[E <= 0.0] = 1
E[E > 8768] = 17000

In [ ]:
p = p*10000
p.astype(np.ushort).tofile(pname)
E.astype(np.ushort).tofile(Ename)